## Lab 02: Integration with LangChain

### We can use Actions to integrate LangChain.


In [1]:
## setup OPENAI_API_KEY

import os
from pprint import pp
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')


We can also use variables in combination with if/else statements to make the behavior more dynamic

In [2]:
from nemoguardrails import LLMRails, RailsConfig

In [3]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
# - type: embeddings
#   engine: openai
#   model: text-embedding-ada-002
"""

In [4]:
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
      bot personal greeting
    else
        bot express greeting

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
rails = LLMRails(config=config)

/Users/jimmy.liao/.pyenv/versions/3.10.13/envs/guardrails/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# vectorstore = Chroma.from_texts(texts=["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
#                                  "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
#                                  "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
#                                  "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
#                                  "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
#                                 embedding=OpenAIEmbeddings())
vectorstore = Chroma.from_texts(texts=["CatBank's Furry Paws Credit Card offers 2% cashback on pet stores and 1% on all other purchases.",
                                 "Meow Loans at CatBank come with a competitive 5% APR, subject to your catnip credit score.",
                                 "Earn up to 4% interest with CatBank's Kitty Savers account, with a minimum deposit of $500.",
                                 "Invest in the Whisker Growth Fund with a minimum of $1,000 and enjoy purr-tential annual returns of 7%.",
                                 "Open a Feline Secure savings account with just $100; enjoy up to 3% interest with no monthly fees."],
                                embedding=OpenAIEmbeddings()
                                )

retriever = vectorstore.as_retriever()
prompt_template = """You are a helpful bot for our Bank. Only answer if your have got content in "Context".
Otherwise tell the user in a friendly way that you do not know and can not help with that.

Context: {context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=rails.llm, # Use llm attached to rail instance
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
)

In [7]:
rails.register_action(qa, name="qa_chain")

In [8]:
history = [{"role": "user","content": "Whats the minimum deposit for CatBank's Kitty Savers account?"}]
bot_message = await rails.generate_async(messages=history)
print(bot_message)

{'role': 'assistant', 'content': " The minimum deposit for CatBank's Kitty Savers account is $500."}


In [9]:
history = [{"role": "user","content": "Make a joke about a cow"}]
bot_message = await rails.generate_async(messages=history)
print(bot_message)

{'role': 'assistant', 'content': " I'm sorry, I don't know how to help you with that."}


We normally don´t want to our bot to answer to questions not related to our business topics, so we add Guardrails which cover specific off topic questions and also a chit-chat category

In [10]:
new_colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you with your cat-related banking today?"

define bot personal greeting
    "Hello $username, nice to see you again! Hope your cat's doing well!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting

define user ask about dogs
    "Can I get a loan for my dog?"
    "Do you provide pet insurance for dogs?"
    "What about doggie credit cards?"

define bot no dogs policy
    "Here at CatBank, we're all about cats! We don't offer services for dogs, but we can help with any cat-related banking needs."

define flow dog policy
    user ask about dogs
    bot no dogs policy

define user silly cat question
    "Can my cat open its own bank account?"
    "Do you think cats are better bankers than humans?"
    "Can I use a picture of my cat as my credit card design?"

define bot respond to silly cat question
    "While we love your cat's enthusiasm, only humans can open bank accounts. But we totally agree that cats would make amazing bankers, and yes, your credit card can definitely feature your cat's majestic portrait!"

define flow silly cat questions
    user silly cat question
    bot respond to silly cat question

define user chit chat
    "What do you think about the latest movie?"
    "Got any weekend plans?"
    "Can you tell me a joke?"

define bot finance only
    "Sorry, I do cat related finance advice only!"

define flow chit chat
    user chit chat
    bot finance only

define flow
    user ...
    $answer = execute qa_chain(query=$last_user_message)
    bot $answer
"""

In [11]:
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=new_colang_content
)
rails = LLMRails(config=config)
rails.register_action(qa, name="qa_chain")

In [12]:
messages = [
    {"role": "context", "content": {"username": "Markus"}},
    {"role": "user", "content": "Hello!"},
]
res = await rails.generate_async(messages=messages)
print(res)

{'role': 'assistant', 'content': "Hello Markus, nice to see you again! Hope your cat's doing well!"}


In [13]:
messages = [{"role": "user","content": "Can you tell me a joke?"}]
bot_message = await rails.generate_async(messages=messages)
print(bot_message)

{'role': 'assistant', 'content': 'Sorry, I do cat related finance advice only!'}
